# Get LLDP informations

### 首先，把case2 中用于登录设备的代码写在一个Device.py中，用于登录设备。
Device.py文件位于这个case3目录中。 我们可以打开这个文件查看其内容

### 其次，写一个用于IOSXR的类，用于获取设备的lldp信息。
在方法 get_lldp中 还有一个function 用于处理 lldp的输出信息为一个格式化后的格式。而非纯文本信息

### 最后，使用这个新的class 通过asyncio的方式来获取设备的lldp信息




In [1]:

from Device import Device
from pexpect import EOF, TIMEOUT

In [4]:
class IOSXR(Device):
    
    def __init__(self, device):
        super(IOSXR, self).__init__(device)
        self.prompt = self.hostname + "[>|#]\s?"

    async def login(self, prompt=""):
        if not prompt:
            prompt = self.prompt
        await super(IOSXR, self).login(prompt)

    async def _set_terminal_length_zero(self):
        self.c.sendline("terminal length 0")
        try:
            i = await self.c.expect(self.prompt, async_=True)
        except EOF:
            pass
        except TIMEOUT:
            print("session timeout")

    async def get_lldp(self):
        
        # parser cli's lldp output. the result is a list.
        def _parser_lldp(lldp_lines):
            start_flag = 0
            lldp_neighbors = []
            for line in lldp_lines.splitlines():
                line = line.strip()
                if "Device ID" in line:
                    start_flag = 1
                    continue
                if start_flag == 1 and len(line) == 0:
                    break
                if start_flag == 1:
                    line = line.split()
                    lldp_neighbors.append({"device_id":line[0],  
                                            "Local_intf":line[1],
                                            "Port_id":line[-1]})
            return lldp_neighbors
        
        self.expect_list = []
        self.expect_list.append(self.prompt)
        result = []
        self.c.sendline("show lldp neighbors")
        try:
            i = await self.c.expect(self.expect_list, timeout=5, async_=True)
            if i == 0:
                lldp_lines = str(self.c.before, 'utf-8')
                result = _parser_lldp(lldp_lines)
        except EOF:
            pass
        except TIMEOUT:
            print("session timeout")
        return result


## 在协程中获取返回值
### 为了方便的获取每个协程的返回值。我们需要为从每台设备上获取信息的方法创建一个task。最后在task中来获取返回值。

创建一个任务（task）
```
asyncio.ensure_future(get_lldp(xrv1))
```

然后把这些任务放在一个list中。

### 使用event_loop来执行这些tasks

```
loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)

# create a tasks list
tasks = []
tasks.append(asyncio.ensure_future(get_lldp(xrv1)))

loop.run_until_complete(asyncio.wait(tasks))

```

In [16]:
import asyncio


async def get_lldp(device):
    con = IOSXR(device)
    con.connect()
    await con.login()
    result = await con.get_lldp()
#     print(parser_lldp(result[1]))
    return {'hostname': con.hostname, 'lldp_info': result}

xrv1 = {"hostname":"xrv1", 
        "mgt_ip":"172.20.3.101",
        "username":"admin",
        "password":"admin",
        "port": 22}

xrv2 = {"hostname":"xrv2", 
        "mgt_ip":"172.20.3.102",
        "username":"admin",
        "password":"admin",
        "port": 22}

xrv3 = {"hostname":"xrv3", 
        "mgt_ip":"172.20.3.103",
        "username":"admin",
        "password":"admin",
        "port": 22}

xrv4 = {"hostname":"xrv4", 
        "mgt_ip":"172.20.3.104",
        "username":"admin",
        "password":"admin",
        "port": 22}

loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)

tasks = []
tasks.append(asyncio.ensure_future(get_lldp(xrv1)))
tasks.append(asyncio.ensure_future(get_lldp(xrv2)))
tasks.append(asyncio.ensure_future(get_lldp(xrv3)))
tasks.append(asyncio.ensure_future(get_lldp(xrv4)))


loop.run_until_complete(asyncio.wait(tasks))

({<Task finished coro=<get_lldp() done, defined at <ipython-input-16-efe5d11fee34>:4> result={'hostname': 'xrv1', 'lldp_info': [{'Local_intf': 'Gi0/0/0/0', 'Port_id': 'Gi0/0/0/0', 'device_id': 'xrv2'}, {'Local_intf': 'Gi0/0/0/1', 'Port_id': 'Gi0/0/0/0', 'device_id': 'xrv3'}, {'Local_intf': 'Gi0/0/0/4', 'Port_id': 'Gi0/0/0/2', 'device_id': 'xrv6'}, {'Local_intf': 'Gi0/0/0/5', 'Port_id': 'Gi0/0/0/2', 'device_id': 'xrv7'}]}>,
  <Task finished coro=<get_lldp() done, defined at <ipython-input-16-efe5d11fee34>:4> result={'hostname': 'xrv2', 'lldp_info': [{'Local_intf': 'Gi0/0/0/0', 'Port_id': 'Gi0/0/0/0', 'device_id': 'xrv1'}, {'Local_intf': 'Gi0/0/0/3', 'Port_id': 'Gi0/0/0/1', 'device_id': 'xrv5'}, {'Local_intf': 'Gi0/0/0/4', 'Port_id': 'Gi0/0/0/3', 'device_id': 'xrv6'}, {'Local_intf': 'Gi0/0/0/5', 'Port_id': 'Gi0/0/0/3', 'device_id': 'xrv7'}]}>,
  <Task finished coro=<get_lldp() done, defined at <ipython-input-16-efe5d11fee34>:4> result={'hostname': 'xrv3', 'lldp_info': [{'Local_intf': 'Gi

### 可以从tasks 中获取function返回的值。

In [15]:
for x in tasks:
    print(x.result())

{'lldp_info': [{'Local_intf': 'Gi0/0/0/0', 'Port_id': 'Gi0/0/0/0', 'device_id': 'xrv2'}, {'Local_intf': 'Gi0/0/0/1', 'Port_id': 'Gi0/0/0/0', 'device_id': 'xrv3'}, {'Local_intf': 'Gi0/0/0/4', 'Port_id': 'Gi0/0/0/2', 'device_id': 'xrv6'}, {'Local_intf': 'Gi0/0/0/5', 'Port_id': 'Gi0/0/0/2', 'device_id': 'xrv7'}], 'hostname': 'xrv1'}
{'lldp_info': [{'Local_intf': 'Gi0/0/0/0', 'Port_id': 'Gi0/0/0/0', 'device_id': 'xrv1'}, {'Local_intf': 'Gi0/0/0/1', 'Port_id': 'Gi0/0/0/1', 'device_id': 'xrv3'}, {'Local_intf': 'Gi0/0/0/4', 'Port_id': 'Gi0/0/0/3', 'device_id': 'xrv6'}, {'Local_intf': 'Gi0/0/0/5', 'Port_id': 'Gi0/0/0/3', 'device_id': 'xrv7'}], 'hostname': 'xrv2'}
{'lldp_info': [{'Local_intf': 'Gi0/0/0/0', 'Port_id': 'Gi0/0/0/1', 'device_id': 'xrv1'}, {'Local_intf': 'Gi0/0/0/6', 'Port_id': 'Gi0/0', 'device_id': 'vIOS16.sha-se.cisco.'}, {'Local_intf': 'Gi0/0/0/7', 'Port_id': 'Gi0/1', 'device_id': 'vIOS17.sha-se.cisco.'}], 'hostname': 'xrv3'}
{'lldp_info': [{'Local_intf': 'Gi0/0/0/1', 'Port_id': 

# 作业：
## 1. 修改 class IOSXR。 适配IOS设备。
## 2. 把所有登录信息写在一个yaml文件中，读取文件来登录设备。
## 3. 把tasks 中的结果合并到一个字典中。这个字典可以用于后续的路径计算的输入变量。
